<a href="https://colab.research.google.com/github/ahakobia/Group4_NFLX_MIDTERM/blob/main/Adehs_US_Energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np

In [20]:
energy_data = pd.read_csv("Resources/organised_Gen.csv")
energy_data = energy_data.drop(columns=['Unnamed: 0'])
energy_data.head(25)

,YEAR,MONTH,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours)
0,2001,1,AK,Total Electric Power Industry,Coal,46903.0
1,2001,1,AK,Total Electric Power Industry,Petroleum,71085.0
2,2001,1,AK,Total Electric Power Industry,Natural Gas,367521.0
3,2001,1,AK,Total Electric Power Industry,Hydroelectric Conventional,104549.0
4,2001,1,AK,Total Electric Power Industry,Wind,87.0
5,2001,1,AK,Total Electric Power Industry,Total,590145.0
6,2001,1,AK,"Electric Generators, Electric Utilities",Coal,18410.0
7,2001,1,AK,"Electric Generators, Electric Utilities",Petroleum,64883.0
8,2001,1,AK,"Electric Generators, Electric Utilities",Natural Gas,305277.0
9,2001,1,AK,"Electric Generators, Electric Utilities",Hydroelectric Conventional,104549.0


In [41]:
energy_data.head(9000)

,YEAR,MONTH,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours),Total
0,2001,1,AK,Total Electric Power Industry,Coal,46903.0,6.921969e+11
1,2001,1,AK,Total Electric Power Industry,Petroleum,71085.0,6.921969e+11
2,2001,1,AK,Total Electric Power Industry,Natural Gas,367521.0,6.921969e+11
3,2001,1,AK,Total Electric Power Industry,Hydroelectric Conventional,104549.0,6.921969e+11
4,2001,1,AK,Total Electric Power Industry,Wind,87.0,6.921969e+11
...,...,...,...,...,...,...,...
8995,2001,6,MD,"Electric Generators, Independent Power Producers",Total,4166191.0,0.000000e+00
8996,2001,6,MD,"Combined Heat and Power, Electric Power",Coal,110685.0,0.000000e+00
8997,2001,6,MD,"Combined Heat and Power, Electric Power",Petroleum,9535.0,0.000000e+00
8998,2001,6,MD,"Combined Heat and Power, Electric Power",Natural Gas,50882.0,0.000000e+00
